In [5]:
"""
    EXTRACT DATA FROM STOCKTWITS API
    WORKAROUND RATE LIMITS USING PROXY
    CHANGED WORKAROUND METHOD TO USING MULTIPLE ACCESS KEYS
"""

import csv
import json
import os
import time
import requests
import random
import yfinance as yf


# from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

FIELDS = ['symbol', 'sentiment','message', 'datetime', 'user','username','userfollowers','user_likecount','message_id','message_likes']
SYMBOL = ["TSLA","AAPL","GOOG","CHGG","DBX","DNKN","FB","TWTR","MSFT","SNAP","TTM","RS","NFLX","AMZN","JPM","GS","MU"]
#FILE_NAME = 'stocktwits_' + SYM + '.csv'
FILE_NAME = 'stocktwits_scraped4.csv'
token = 0
access_token = ['', 'access_token=32a3552d31b92be5d2a3d282ca3a864f96e95818&',
                'access_token=44ae93a5279092f7804a0ee04753252cbf2ddfee&',
                'access_token=990183ef04060336a46a80aa287f774a9d604f9c&']

file = open(FILE_NAME, 'a', newline='', encoding='utf-8')
# DETERMINE WHERE TO START IF RESUMING SCRIPT
if os.stat(FILE_NAME).st_size == 0:
    # OPEN FILE IN APPEND MODE AND WRITE HEADERS TO FILE
    last_message_id = None
    csvfile = csv.DictWriter(file, FIELDS)
    csvfile.writeheader()
else:
    # FIRST EXTRACT LAST MESSAGE ID THEN OPEN FILE IN APPEND MODE WITHOUT WRITING HEADERS
    file = open(FILE_NAME, 'r', newline='', encoding='utf-8')
    csvfile = csv.DictReader((line.replace('\0', '') for line in file))
    data = list(csvfile)
    data = data[-1]
    last_message_id = data['message_id']
    file.close()
    file = open(FILE_NAME, 'a', newline='', encoding='utf-8')
    csvfile = csv.DictWriter(file, FIELDS)

# req_proxy = RequestProxy()

SYM = "TWTR"

stocktwit_url = "https://api.stocktwits.com/api/2/streams/symbol/" + SYM + ".json?" + access_token[token]



if last_message_id is not None:
    stocktwit_url += "max=" + str(last_message_id)

count = 0
api_hits = 0
while api_hits < 200:
    # response = req_proxy.generate_proxied_request(stocktwit_url)
    try:
        response = requests.get(stocktwit_url)
    except Exception:
        response = None

    if response is not None:

        if response.status_code == 429:
            print("###############")
            print("REQUEST IP RATE LIMITED FOR {} seconds !!!".format(
                int(response.headers['X-RateLimit-Reset']) - int(time.time()))) #int(time.time()) = 1577817000

        if not response.status_code == 200:
            #print(last_message_id)
            stocktwit_url = "https://api.stocktwits.com/api/2/streams/symbol/" + SYM + ".json?" + access_token[
                token] + "max=" + str(
                last_message_id)
            token = (token + 1) % (len(access_token))
            continue

        api_hits += 1
        #print(response.text)
        response = json.loads(response.text)
        last_message_id = response['cursor']['max']
        
        #print("Last Message ID: ",last_message_id)
        #print("response")
        # WRITE DATA TO CSV FILE
        for message in response['messages']:
            # PREPARE OBJECT TO WRITE IN CSV FILE
            #print(message)
            temp = message['entities']['sentiment']
            if temp is not None and temp['basic']:
                obj = {}
                obj['symbol'] = SYM
                obj['sentiment'] = temp['basic']
                obj['message_id'] = message['id']
                obj['message'] = message['body']
                obj['datetime'] = message['created_at']
                obj['user'] = message['user']['id']
                obj['username'] = message['user']['username']
                obj['userfollowers'] = message['user']['followers']
                #obj['userfollowing'] = message['user']['following']
                obj['user_likecount'] = message['user']['like_count']
            
                csvfile.writerow(obj)
                file.flush()
                print("Record Number: ",count)
                count += 1

        print("API HITS TILL NOW = {}".format(api_hits))

        # NO MORE MESSAGES
        if not response['messages']:
            break

    # ADD MAX ARGUMENT TO GET OLDER MESSAGES
    SYM = random.choice(SYMBOL)
    stocktwit_url = "https://api.stocktwits.com/api/2/streams/symbol/" + SYM + ".json?" + access_token[
        token] + "max=" + str(last_message_id)
    token = (token + 1) % (len(access_token))

file.close()

Record Number:  0
Record Number:  1
Record Number:  2
Record Number:  3
Record Number:  4
Record Number:  5
Record Number:  6
Record Number:  7
Record Number:  8
Record Number:  9
Record Number:  10
API HITS TILL NOW = 1
Record Number:  11
Record Number:  12
Record Number:  13
Record Number:  14
Record Number:  15
Record Number:  16
Record Number:  17
Record Number:  18
Record Number:  19
Record Number:  20
Record Number:  21
Record Number:  22
Record Number:  23
Record Number:  24
Record Number:  25
Record Number:  26
Record Number:  27
Record Number:  28
API HITS TILL NOW = 2
Record Number:  29
Record Number:  30
Record Number:  31
Record Number:  32
Record Number:  33
Record Number:  34
Record Number:  35
Record Number:  36
Record Number:  37
Record Number:  38
Record Number:  39
Record Number:  40
Record Number:  41
Record Number:  42
API HITS TILL NOW = 3
Record Number:  43
Record Number:  44
API HITS TILL NOW = 4
Record Number:  45
Record Number:  46
Record Number:  47
Record Num

Record Number:  378
Record Number:  379
API HITS TILL NOW = 34
Record Number:  380
Record Number:  381
Record Number:  382
Record Number:  383
API HITS TILL NOW = 35
API HITS TILL NOW = 36
Record Number:  384
Record Number:  385
Record Number:  386
Record Number:  387
Record Number:  388
Record Number:  389
Record Number:  390
Record Number:  391
Record Number:  392
Record Number:  393
Record Number:  394
Record Number:  395
API HITS TILL NOW = 37
Record Number:  396
Record Number:  397
Record Number:  398
Record Number:  399
Record Number:  400
Record Number:  401
Record Number:  402
API HITS TILL NOW = 38
Record Number:  403
Record Number:  404
Record Number:  405
Record Number:  406
Record Number:  407
Record Number:  408
Record Number:  409
Record Number:  410
Record Number:  411
API HITS TILL NOW = 39
Record Number:  412
Record Number:  413
Record Number:  414
Record Number:  415
Record Number:  416
Record Number:  417
Record Number:  418
Record Number:  419
Record Number:  420
AP

Record Number:  753
Record Number:  754
Record Number:  755
Record Number:  756
Record Number:  757
Record Number:  758
Record Number:  759
Record Number:  760
Record Number:  761
Record Number:  762
API HITS TILL NOW = 70
Record Number:  763
Record Number:  764
Record Number:  765
Record Number:  766
Record Number:  767
Record Number:  768
Record Number:  769
Record Number:  770
Record Number:  771
Record Number:  772
Record Number:  773
Record Number:  774
Record Number:  775
Record Number:  776
Record Number:  777
API HITS TILL NOW = 71
Record Number:  778
Record Number:  779
Record Number:  780
Record Number:  781
Record Number:  782
Record Number:  783
Record Number:  784
Record Number:  785
Record Number:  786
Record Number:  787
API HITS TILL NOW = 72
Record Number:  788
Record Number:  789
Record Number:  790
Record Number:  791
Record Number:  792
Record Number:  793
Record Number:  794
Record Number:  795
Record Number:  796
API HITS TILL NOW = 73
Record Number:  797
Record N

Record Number:  1128
Record Number:  1129
Record Number:  1130
Record Number:  1131
Record Number:  1132
Record Number:  1133
Record Number:  1134
Record Number:  1135
Record Number:  1136
API HITS TILL NOW = 106
Record Number:  1137
Record Number:  1138
Record Number:  1139
Record Number:  1140
Record Number:  1141
Record Number:  1142
Record Number:  1143
Record Number:  1144
Record Number:  1145
Record Number:  1146
Record Number:  1147
Record Number:  1148
Record Number:  1149
Record Number:  1150
Record Number:  1151
Record Number:  1152
Record Number:  1153
API HITS TILL NOW = 107
Record Number:  1154
Record Number:  1155
Record Number:  1156
API HITS TILL NOW = 108
Record Number:  1157
Record Number:  1158
Record Number:  1159
Record Number:  1160
Record Number:  1161
Record Number:  1162
Record Number:  1163
Record Number:  1164
API HITS TILL NOW = 109
Record Number:  1165
Record Number:  1166
Record Number:  1167
Record Number:  1168
Record Number:  1169
Record Number:  1170
A

Record Number:  1482
Record Number:  1483
Record Number:  1484
Record Number:  1485
Record Number:  1486
Record Number:  1487
Record Number:  1488
Record Number:  1489
API HITS TILL NOW = 141
Record Number:  1490
Record Number:  1491
Record Number:  1492
Record Number:  1493
Record Number:  1494
Record Number:  1495
Record Number:  1496
Record Number:  1497
Record Number:  1498
Record Number:  1499
API HITS TILL NOW = 142
Record Number:  1500
Record Number:  1501
Record Number:  1502
Record Number:  1503
Record Number:  1504
Record Number:  1505
API HITS TILL NOW = 143
Record Number:  1506
Record Number:  1507
Record Number:  1508
Record Number:  1509
Record Number:  1510
Record Number:  1511
Record Number:  1512
Record Number:  1513
Record Number:  1514
Record Number:  1515
Record Number:  1516
Record Number:  1517
Record Number:  1518
Record Number:  1519
Record Number:  1520
Record Number:  1521
API HITS TILL NOW = 144
Record Number:  1522
Record Number:  1523
Record Number:  1524
R

Record Number:  1834
Record Number:  1835
Record Number:  1836
Record Number:  1837
Record Number:  1838
Record Number:  1839
Record Number:  1840
Record Number:  1841
Record Number:  1842
Record Number:  1843
Record Number:  1844
Record Number:  1845
Record Number:  1846
Record Number:  1847
Record Number:  1848
API HITS TILL NOW = 176
Record Number:  1849
Record Number:  1850
Record Number:  1851
Record Number:  1852
Record Number:  1853
Record Number:  1854
API HITS TILL NOW = 177
Record Number:  1855
API HITS TILL NOW = 178
Record Number:  1856
Record Number:  1857
Record Number:  1858
Record Number:  1859
Record Number:  1860
API HITS TILL NOW = 179
Record Number:  1861
Record Number:  1862
Record Number:  1863
Record Number:  1864
Record Number:  1865
Record Number:  1866
Record Number:  1867
Record Number:  1868
Record Number:  1869
Record Number:  1870
Record Number:  1871
Record Number:  1872
Record Number:  1873
API HITS TILL NOW = 180
Record Number:  1874
Record Number:  187

185244928

In [12]:
import time
seconds = int(time.time())
seconds

1603678722

In [13]:
local_time = time.ctime(seconds)
print("Local time:", local_time)

Local time: Mon Oct 26 07:48:42 2020


In [14]:
local_time = time.ctime(1577817000)
print("Local time:", local_time)

Local time: Wed Jan  1 00:00:00 2020


In [19]:
local_time = time.ctime(1593541799)
print("Local time:", local_time)

Local time: Tue Jun 30 23:59:59 2020


In [46]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 1.7MB/s eta 0:00:01
  Stored in directory: /home/prem/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
  Stored in directory: /home/prem/.cache/pip/wheels/37/fa/73/d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
  Found existing installation: lxml 4.3.4
    Uninstalling lxml-4.3.4:
      Successfully uninstalled lxml-4.3.4


In [47]:
import yfinance as yf

In [49]:
msft = yf.Ticker("MSFT")
msft

yfinance.Ticker object <MSFT>

In [97]:
try:
    price = (yf.download('AAPL','2020-01-01','2020-01-01',rounding=True)['Adj Close']).values[0]
except Exception:
    price = None
    
print(price)

[*********************100%***********************]  1 of 1 completed
72.91


In [95]:
print(data)

None


In [81]:
da_dict = {}
da_dict['price'] = data.values[0]

In [82]:
data.values

array([72.90950012])

In [84]:
da_dict['price']

72.90950012207031

In [85]:
time = '2020-10-26T04:56:36Z'

In [87]:
time[0:10]

'2020-10-26'

In [98]:
price = yf.download('AAPL','2020-01-01','2020-01-31',rounding=True)

[*********************100%***********************]  1 of 1 completed


In [99]:
price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,72.48,73.42,72.38,73.41,72.91,100805600
2020-01-02,74.06,75.15,73.80,75.09,74.57,135480400
2020-01-03,74.29,75.14,74.12,74.36,73.85,146322800
2020-01-06,73.45,74.99,73.19,74.95,74.44,118387200
2020-01-07,74.96,75.22,74.37,74.60,74.09,108872000
2020-01-08,74.29,76.11,74.29,75.80,75.28,132079200
2020-01-09,76.81,77.61,76.55,77.41,76.88,170108400
2020-01-10,77.65,78.17,77.06,77.58,77.05,140644800
2020-01-13,77.91,79.27,77.79,79.24,78.70,121532000
